In [10]:
import os
import json
import pandas as pd
import traceback


In [11]:
from langchain.chat_models import ChatOpenAI

In [12]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
KEY= os.getenv("OPENAI_API_KEY")

In [14]:
print(KEY)

sk-None-cP8uS2dq5inseAywgksBT3BlbkFJxH7yPYXt3ID8z8ynRYT7


In [15]:
llm = ChatOpenAI(openai_api_key = KEY,model_name = "gpt-3.5-turbo",temperature = 0.5)

In [16]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D7FC2F7110>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D7FC320E30>, temperature=0.5, openai_api_key='sk-None-cP8uS2dq5inseAywgksBT3BlbkFJxH7yPYXt3ID8z8ynRYT7', openai_proxy='')

In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

In [23]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },

    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [24]:
TEMPLATE = """
  Text:{text}
  You are an expert MCQ maker. Given the above text, it is your job to 
  create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
  Make sure the questions are not repeated and check all the questions to be conforming the text as well.
  Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
  Ensure to make {number} MCQS 
  ### RESPONSE_JSON
  {response_json}

"""

In [26]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [27]:
quiz_chain = LLMChain(llm=llm, prompt = quiz_generation_prompt, output_key="quiz",verbose=True)

In [28]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for coplexity 
if the quiz is not per with the cognitive and analytical ability of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilites 
Quiz_MCQS:

{quiz}

Check from an expert English writer of the above quiz: 

"""

In [29]:
quiz_evaluation_prompt = PromptTemplate(
input_variables = ["subject","quiz"],
template = TEMPLATE2
)

In [30]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt, output_key="review", verbose=True)

In [31]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                          output_variables = ["quiz","review"],verbose = True)

In [32]:
# for reading the data.txt file

In [35]:
file_path = r"F:\mcqgenPRJ\src\experiment\data.txt"


In [36]:
file_path

'F:\\mcqgenPRJ\\src\\experiment\\data.txt'

In [37]:
with open (file_path,'r') as file:
    TEXT = file.read()

In [38]:
TEXT

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contribu

In [39]:
#to serizlize python dictionary into a JSON formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [41]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [42]:

# how to setup token usage tracking in langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used

In [43]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens:1961
Prompt Tokens:1463
Completion Tokens:498
Total Cost: 0.0031905


In [44]:
#getting a response
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [45]:
quiz = response.get("quiz")

In [47]:
quiz = json.loads(quiz)

In [48]:
quiz

{'1': {'mcq': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'options': {'a': 'Artificial Intelligence',
   'b': 'Machine Learning',
   'c': 'Neural Networks',
   'd': 'Deep Learning'},
  'correct': 'b'},
 '2': {'mcq': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 '3': {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'Cyberdyne',
   'b': 'Cybernet',
   'c': 'Cyberspace',
   'd': 'Cybertron'},
  'correct': 'd'},
 '4': {'mcq': 'According to Tom M. Mitchell, when is a computer program said to learn from experience?',
  'options': {'a': 'When it can think independently',
   'b': 'When it can perform tasks better with experience',
   'c': 

In [49]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [50]:
quiz_table_data

[{'MCQ': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a:Artificial Intelligence | b:Machine Learning | c:Neural Networks | d:Deep Learning',
  'Correct': 'b'},
 {'MCQ': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'Choices': 'a:Arthur Samuel | b:Donald Hebb | c:Walter Pitts | d:Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a:Cyberdyne | b:Cybernet | c:Cyberspace | d:Cybertron',
  'Correct': 'd'},
 {'MCQ': 'According to Tom M. Mitchell, when is a computer program said to learn from experience?',
  'Choices': 'a:When it can think independently | b:When it can perform tasks better with experience | c:When it can predict the future accurately | d:When it can understand human emotions',
  'Correct': 'b'},
 {'MCQ': '

In [52]:
# converting to a df
quiz = pd.DataFrame(quiz_table_data)

In [53]:
quiz.to_csv("machinelearningquiz.csv",index=False)

In [4]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_28_2024_11_08_23'